In [1]:
import pandas as pd 

In [4]:
planillas_hist=pd.read_csv('vehiculos_sin_modelo.csv',index_col=0)

# Importar librerias

In [6]:
import time 

In [7]:
import requests

In [8]:
from tqdm.notebook import tqdm

In [9]:
from bs4 import BeautifulSoup

# Definir funciones

Request detalles de revision

In [10]:
def get_detalles(nro,codTaller,versionRTO):#input is in string format 
    global soup
    url = "https://rto.cent.gov.ar/rto/RTO/verDetallePlanillaRevision"

    payload = "nroPlanilla="+nro+"&codTaller="+codTaller+"&versionRTO="+versionRTO
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0",
        "Accept": "text/javascript, text/html, application/xml, text/xml, */*",
        "Accept-Language": "es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "X-Requested-With": "XMLHttpRequest",
        "X-Prototype-Version": "1.6.1",
        "Content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://rto.cent.gov.ar",
        "Connection": "keep-alive",
        "Referer": "https://rto.cent.gov.ar/rto/RTO/planillaDiaria",
        "Cookie": cookie,
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    #print response text (test)
    #print(response.text)
    soup=BeautifulSoup(response.text,'lxml')

Limpiar soup para detalles 

In [11]:
def clean_data(tag):# returns a list of values: tags
        tags_datos=soup.select(tag) #data is inside the tag td
        global datos
        datos=[] 
        for data in tags_datos:
            datos.append(data.get_text(strip=True)) #make a list with the values 
        return(datos)

Limpiar soup para anomalias 

In [12]:
def clean_data_anomalias(tag_anomalias):# returns a list of values: tags
        tags_datos_anomalias=soup.select(tag_anomalias) #data is inside the tag td
        global datos_anomalias
        datos_anomalias=[] 
        for data in tags_datos_anomalias:
            datos_anomalias.append(data.get_text(strip=True)) #make a list with the values 
        return(datos_anomalias)

Crear lista de diccionarios con los detalles: cada diccionario es una fila 

In [13]:
def create_rows_detalles():
    global df
    global rows
    #make dictionary with only the relevant elements 
    #Search Label in the list,if it doesnt exist, returns empty string 
    if 'Número:'in datos:
        Planilla=datos[datos.index('Número:')+1]
    else:
        Planilla=''
    if 'Dominio:'in datos:
        Dominio=datos[datos.index('Dominio:')+1]
    else:
        Dominio=''
    if 'Año Modelo:'in datos:
        Modelo=datos[datos.index('Año Modelo:')+1]
    else:
        Modelo=''
    if 'Convenio:'in datos:
        Convenio=datos[datos.index('Convenio:')+1]
    else:
        Convenio=''
    if 'Resultado:'in datos:
        Resultado=datos[datos.index('Resultado:')+1]
    else:
        Resultado=''


    rows.append({'Planilla':Planilla,
     'Dominio:':Dominio,
     'Modelo':Modelo,
     'Convenio:':Convenio,
     'Resultado':Resultado
    })#make dict 

Crear lista de anomalias:Para cada anomalia es una fila

In [14]:
def create_rows_anomalias(nro_planilla):
    global df_anomalias
    global rows_anomalias
    if len(datos_anomalias)>0:
        a=int(0)
        for i in range (0,int((len(datos_anomalias))/4)):
            rows_anomalias.append([nro_planilla,datos_anomalias[a],datos_anomalias[a+1],datos_anomalias[a+2],datos_anomalias[a+3]])
            a+=int(4)
    else:
        rows_anomalias.append([nro_planilla,None,None,None,None])

Crear dataframe (df) de detalles 

In [15]:
def create_df_detalles():# returns a dataframe df 
    global df 
    df=pd.DataFrame(rows)
    return(df)

Crear dataframe (df_anomalias) de anomalias

In [16]:
def create_df_anomalias():# returns a dataframe df 
    global columns_anomalias
    global df_anomalias 
    columns_anomalias=['Planilla','Código','Gravedad','Descripción','Observaciones']
    df_anomalias=pd.DataFrame()
    df_anomalias=pd.DataFrame(rows_anomalias,columns=columns_anomalias)
    return(df_anomalias)

Utilizando las funciones anteriores. las unimos en una unica funcion. Iteramos en la lista de Nroplanillas y nroTaller

#  FUNCION SCRAPE

In [17]:
def scrape(lista_planillas):
    global rows
    global rows_anomalias
    rows=[]
    rows_anomalias=[] 
    tag='label'#tag detalles planilla  
    tag_anomalias='td'#tag anomalias
    for i in tqdm(lista_planillas):
        get_detalles(str(i[0]),str(i[1]),str(i[2]))
        clean_data(tag)
        create_rows_detalles()
        clean_data_anomalias(tag_anomalias)
        create_rows_anomalias(i[0])
    create_df_detalles()# returns dataframe df
    create_df_anomalias()#returns dataframe df_anomalias de anomalias 
    return([df['Planilla'].nunique(),df_anomalias['Planilla'].nunique()])

In [18]:
def scrape_test(lista_planillas):
    global rows
    global rows_anomalias
    rows=[]
    rows_anomalias=[] 
    tag='label'#tag detalles planilla  
    tag_anomalias='td'#tag anomalias
    for nro_planilla in tqdm(lista_planillas):
        get_detalles(nro_planilla[0],str(nro_planilla[1]))
        clean_data(tag)
        create_rows_detalles()
        clean_data_anomalias(tag_anomalias)
        create_rows_anomalias(nro_planilla[0])
    create_df_detalles()# returns dataframe df
    create_df_anomalias()#returns dataframe df_anomalias de anomalias 
    return([df['Planilla'].nunique(),df_anomalias['Planilla'].nunique()])

Guardar los dataframes en archivos .csv

In [44]:
def to_csv(num):
    filename=str(num)+'planillas.detalle.faltante'
    df.to_csv(filename)
    filename2=str(num)+'Anomalias.faltante'
    df_anomalias.to_csv(filename2)

Dividimos la lista de planillas en N listas. En caso de error no perdemos todo 

In [45]:
def div_lista(lista,part):
    global lista_part
    n=int(len(lista)/part)
    lista_part=[lista[i:i+n]for i in range(0,len(lista),n)]
    return(lista_part)
    
    

Definimos una nueva funcion Scrape que crea un csv por cada particion de la lista de planillas. Esto nos ayuda a no tener que comenzar de 0 si el server se cae o se corta internet 

In [46]:
def scrape_part(lista,part,start):
    div_lista(lista,part)
    for i in tqdm(range(0+start,part)):
        scrape(lista_part[i])
        to_csv(i)


Definimos una funcion para unir los archivos csv en un unico Dataframe y guardar como csv

In [64]:
def join_csv(part):
    global df_det
    global detalles
    global df_anom
    global anomalias
    detalles=pd.DataFrame()
    anomalias=pd.DataFrame()
    for i in range(0,part):
        df_det=pd.read_csv(str(i)+'planillas.detalle.faltante',index_col=0)
        detalles=pd.concat([detalles,df_det])
        df_anom=pd.read_csv(str(i)+'Anomalias.faltante',index_col=0)
        anomalias=pd.concat([anomalias,df_anom])        
    detalles.to_csv('0.0.planillas.detalle.COMPLETE.faltante.csv')
    anomalias.to_csv('0.0.Anomalias.COMPLETE.faltante.csv')
    


# Actualizar cookie

In [48]:
cookie="JSESSIONID=3AF3C853D67F329648404C040356212C"

# Scrape completo

In [115]:
planillas_lista=planillas_hist.loc[:,['Planilla','Taller','Versión RTO']] #numero de planillas: 14272
planillas_lista.drop_duplicates(inplace=True)#numero de planillas 14272


In [60]:
planillas_lista=planillas_lista.values.tolist()

In [62]:
len(planillas_lista)

14272

In [92]:
scrape_part(planillas_lista,10,0)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

...

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

  0%|          | 0/1427 [00:00<?, ?it/s]

In [106]:
join_csv(9)

# Verificar Datos Detalles planillas

In [116]:
df_detalles_parcial=pd.read_csv(str(5)+'planillas.detalle.faltante',index_col=0)
df_detalles_parcial.head()

,Planilla,Dominio:,Modelo,Convenio:,Resultado
0,1091341,AA309EZ,2016,Mercosur G (Mercosur-Mercosur),Apto
1,1091349,OVW357,2015,UP Chaco(Provincial-Chaco),Apto
2,1091353,AA304PZ,2016,UP Chaco(Provincial-Chaco),Apto
3,1091373,NJD191,2013,UP Chaco(Provincial-Chaco),Apto
4,1091406,HAG028,2008,UP Chaco(Provincial-Chaco),Apto


# Verificar datos anomalias

In [117]:
df_anomalias_parcial=pd.read_csv(str(5)+'Anomalias.faltante',index_col=0)
df_anomalias_parcial.head()

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1091341,NaN,NaN,NaN,NaN
1,1091353,NaN,NaN,NaN,NaN
2,1091373,Número de Chasis,MAL CARGADO EN RTO ANTERIOR,"Valdez, Sergio (TRT: 004)",Número de Motor
3,1091409,NaN,NaN,NaN,NaN
4,1091476,010401,M,"Sistema de Dirección > Barras, Brazos, Amortig...",EXTREMO IZQ


# Leer csv completo y analizar

detalles

In [118]:
detalles=pd.read_csv('0.0.planillas.detalle.COMPLETE.faltante.csv',index_col=0)


In [122]:
detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12843 entries, 0 to 1426
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Planilla   12843 non-null  int64 
 1   Dominio:   12843 non-null  object
 2   Modelo     12843 non-null  int64 
 3   Convenio:  12843 non-null  object
 4   Resultado  12843 non-null  object
dtypes: int64(2), object(3)
memory usage: 602.0+ KB


anomalias

In [123]:
anomalias=pd.read_csv('0.0.Anomalias.COMPLETE.faltante.csv',index_col=0)
anomalias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22819 entries, 0 to 2194
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Planilla       22819 non-null  int64 
 1   Código         19383 non-null  object
 2   Gravedad       19383 non-null  object
 3   Descripción    19383 non-null  object
 4   Observaciones  6798 non-null   object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB
